# 对ctree文件夹的一些分析

## 代码分析环节

（这个section中请大家撰写对代码的分析，其中的关键点）

1. ctree中主要的文件为cminimax.cpp和cnode.cpp，前者主要用于EfficientZero paper的Backup过程中的公式10，也就是分别协助计算$\min_{(s,a)}Q(s,a)$和$\max_{(s,a)}Q(s,a)$，也就是一些statistic的信息。
1. cminimax.cpp中的`normalize`函数实现了EfficientZero paper中公式10的过程，注意$\epsilon$为0.01，`delta`小于该值的话， 则直接用$\epsilon$。
1. cnode.cpp涵盖了MCTS中的流程中Selection 、Expansion 、Backup。
    1. 从Selection开始：对应代码，可以先观察`cselect_child`函数。如下：
        1. `cucb_score`计算得到了paper中公式7对应的结果（在arg max之前）。但是它维护一个了`max_index_lst`，其特点就是只要是大于等于`max_score - epsilon`都会被加入该list，并在函数的结尾处从该list中随机sample一个action返回。
        1. 考虑`cucb_score`如何计算，见函数`cucb_score`：大的结构为`prior_score + value_score`，其中`value_score`为公式7中$Q(s,a)$项，`prior_score`为剩余的部分。`value_score`的计算较为复杂，其需要考虑到`child->visit_count`为0的情况。`parent_mean_q`对应于公式8的$\hat{Q}(s)$。如果$N(s,a)$不为0，那么，就是一般的$Q(s,a)$，计算方式也就是`true_reward + discount * child->value()`。
        1. 不管何种方式，$Q(s,a)$都需要被normalize，也就是公式10中的做法，相比MuZero，其多了0.01的`delta`的下限。值得注意的是，代码上对`value_score`做了clip到$[0,1]$的操作。
    1. 回到MCTS的Selection部分。因为这里实现的是所谓的batch MCTS，所以，真正的Selection部分为函数`cbatch_traverse`。
    

## 讨论环节

（这个section中，请大家提出对repo中难理解部分的疑问，其他同学或老师看到了会进行解答）

1. this is an example question. posted by lezhang.thu@gmail.com

cnode.cpp文件中，出现了大量的`is_reset`，不知道其语义是什么。